In [19]:
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes
import csv
import operator
import pandas as pd

# codes.csv : ID,Parameter_ID,Name,Description,Number
# languages.csv :ID,Name,Macroarea,Latitude,Longitude,Glottocode,ISO639P3code
# parameters.csv : ID,Name,Description
# values.csv : ID,Language_ID,Parameter_ID,Value,Code_ID,Comment,Source,Contribution_ID

In [20]:

# need to determine if category is binary, categorical, or continuous`
# need a map from (param_id,name) -> number
param_info = dict()
file_path = './wals_dataset.cldf/'
with open(file_path + 'codes.csv', 'r') as codes:
    file_reader = csv.reader(codes, delimiter=',')
    next(file_reader)
    for row in file_reader:
        uuid, param_id, name, desc, num = row
        if param_id not in param_info.keys():
            param_info[param_id] = dict()
        param_info[param_id][name] = int(num)


In [21]:

# TODO: Determine categorical variables

# need to determine what languages to use in dataset
# parse in languages BUT!! all values are words :( so use dict from previous


# might change this datastructure later, but it works well with pandase
data = {"languages": [], "features": [], "values": []}
data_v2 = {}
file_path = './wals_dataset.cldf/'
with open(file_path + 'values.csv', 'r') as codes:
    file_reader = csv.reader(codes, delimiter=',')
    next(file_reader)
    for row in file_reader:
        uuid, lang_id, param_id, val, code_id, comment, source, contrib_id = row
        data['languages'].append(lang_id)
        data['features'].append(param_id)
        data['values'].append(param_info[param_id][val])  # value of variable
        if param_id not in data_v2.keys():
            data_v2[param_id] = dict()
        data_v2[param_id][lang_id] = param_info[param_id][val]

df_2 = pd.DataFrame.from_dict(data_v2)


In [22]:

# take top 40 categories intersection of languages & count
counts = {}
for param in data_v2.keys():
    counts[param] = 0
    for lang in data_v2[param]:
        counts[param] += 1

sorted_counts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
# remove features with <200 languages? <100 languages?


In [23]:

# TODO:remove sparse data


# now count # of languages in all of them
#print(data_v2.keys())


def get_set(lt_num=1000):
    lang_set = set(data_v2['1A']).intersection(set(data_v2['13A']))
    param_set = []
    for x, y in sorted_counts:
        if lang_set is None:
            lang_set = set(data_v2[x].keys())
        else:
            lang_set_post = lang_set.intersection(set(data_v2[x].keys()))
            if len(lang_set_post) < lt_num:
                continue
            else:
                lang_set = lang_set_post
                param_set.append(x)
    return lang_set, param_set


def get_set_better(lt_num=1000, nec_set=['1A', '13A','26A']):
    lang_set = set(data_v2[nec_set[0]].keys())
    param_set = set(data_v2.keys())
    params_out=set()
    for feat in nec_set:
        lang_set = lang_set.intersection(set(data_v2[feat].keys()))
        param_set.remove(feat)
        params_out.add(feat)
    while len(lang_set) > lt_num:
        best_set = {}
        best_feat = {}
        for feat in param_set:
            cur_set = lang_set.intersection((set(data_v2[feat].keys())))
            if len(cur_set) > len(best_set):
                best_set = cur_set
                best_feat = feat
        param_set.remove(best_feat)
        params_out.add(best_feat)
        lang_set = best_set
    return lang_set, params_out



In [24]:
'''
print(len(get_set(1000)[1]))
print(len(get_set(700)[1]))
print(len(get_set(500)[1]))#15 parameters for 500 languages
print(len(get_set(200)[1]))#25 parameters for 200
print(len(get_set(lt_num=100)[1]))#30 parameters for 100 languages?
for i in range(1000,100,-10):
    print(i,len(get_set_better(i,['26A','1A','13A'])[1]))
'''

# 220 languages with 25 parameters seems like a good equlibirium. Let's look at the languages that are there
#but 200 with 31 parameters and the one's I want is better

langs, params = get_set_better(200)

# print(langs)



In [25]:


# need to import our language metadata to understand where these languages are from to get an idea of the distribution


# languages.csv :ID,Name,Macroarea,Latitude,Longitude,Glottocode,ISO639P3code
lang_to_names = dict()
with open(file_path + 'languages.csv', 'r') as codes:
    file_reader = csv.reader(codes, delimiter=',')
    next(file_reader)
    for row in file_reader:
        lang_id, name, macro_area, lat, long, glottocode, iso_code = row
        lang_to_names[lang_id] = name

# looks like a prety good mix of langauges (although mainly euro focused) We can change up the dataset later
'''

for lang in langs:
    print(lang_to_names[lang])
'''

# let's also take a look at what features we get from having 220 languages selected


# parameters.csv : ID,Name,Description

param_id_name = dict()
with open(file_path + 'parameters.csv', 'r') as codes:
    file_reader = csv.reader(codes, delimiter=',')
    next(file_reader)
    for row in file_reader:
        param_id, param_name, descr = row
        param_id_name[param_id] = param_name

'''

for param in params:
    print(param_id_name[param])
# most of these parameters are word order and morphological
'''

fin_dataset = dict()
for param in params:
    fin_dataset[param] = dict()
    for lang in langs:
        fin_dataset[param][lang] = data_v2[param][lang]

final_dataset= pd.DataFrame.from_dict(fin_dataset)
#print(final_dataset)
final_dataset.to_csv("dataset.csv")


#get dataset with top 100 features
def get_set_incl(n_feat=140):
    lang_set = set(data_v2['1A']).intersection(set(data_v2['13A']))
    param_set = []
    for x, y in sorted_counts[:n_feat]:
        if lang_set is None:
            lang_set = set(data_v2[x].keys())
        else:
            lang_set = lang_set.union(set(data_v2[x].keys()))
            param_set.append(x)
    inc_dataset = dict()
    for param in param_set:
        inc_dataset[param] = dict()
    for lang in lang_set:
        nan_count = 0
        out_vec = dict()
        for param in param_set:
            try:
                out_vec[param]=(data_v2[param][lang])
                #inc_dataset[param][lang] = data_v2[param][lang]
            except:
                nan_count+=1
                out_vec[param]=(np.nan)
        if nan_count<40:
            for param in param_set:
                inc_dataset[param][lang] = out_vec[param]

    return pd.DataFrame.from_dict(inc_dataset)

incl_dataset= get_set_incl()
print(incl_dataset)
incl_dataset.to_csv('inc_dataset.csv')




     83A  82A  81A  87A  143F  143E  143A  143G  97A  86A  ...  29A  131A  \
ckr    1    1  1.0  2.0   1.0   4.0   2.0   4.0  2.0  1.0  ...    2   NaN   
arp    2    1  2.0  1.0   1.0   1.0  15.0   4.0  3.0  3.0  ...    3   NaN   
bsq    1    1  1.0  2.0   4.0   1.0   1.0   4.0  2.0  1.0  ...    3   2.0   
war    2    2  4.0  NaN   4.0   1.0   1.0   4.0  NaN  2.0  ...    1   6.0   
kse    1    1  1.0  2.0   4.0   1.0   1.0   4.0  2.0  1.0  ...    1   1.0   
..   ...  ...  ...  ...   ...   ...   ...   ...  ...  ...  ...  ...   ...   
awp    1    1  1.0  1.0   3.0   1.0  14.0   4.0  1.0  1.0  ...    2   6.0   
kho    1    1  1.0  1.0   1.0   4.0   2.0   4.0  1.0  1.0  ...    2   1.0   
hai    1    1  1.0  2.0   2.0   1.0  14.0   4.0  2.0  1.0  ...    1   1.0   
abk    1    1  1.0  2.0   2.0   2.0  11.0   4.0  2.0  1.0  ...    3   2.0   
ind    2    1  2.0  2.0   4.0   1.0   1.0   4.0  4.0  2.0  ...    1   1.0   

     108A  79A  80A  79B  90B  98A  108B  109A  
ckr   2.0  4.0  3.0  5.0  

In [83]:
#add category of type of feature to datasets

#incl_dataset
#final_dataset

#import cats
import os
import csv
vals_to_type=dict()
for filename in os.listdir(os.getcwd()+'/feats/'):
    tp=filename.split(".")[0]
    try:
        fp= open("./feats/"+filename,"r")
    
        file_reader = csv.reader(fp, delimiter=',')
        next(file_reader)
        for row in file_reader:
            uuid=row[3]
            vals_to_type[uuid]=tp
        fp.close()
    except:
        continue
tp_to_count = dict()
for colname,_ in incl_dataset.iteritems():
    try:
        tp = vals_to_type[colname]
        try:
            tp_to_count[tp]+=1
        except:
            tp_to_count[tp]=1
    except:
        print(colname)
print(tp_to_count)
data = incl_dataset.copy()
for colname,_ in incl_dataset.iteritems():
    if vals_to_type[colname] == "lexicon" or vals_to_type[colname] == "phonology" or vals_to_type[colname] == "morphology":
        del data[colname]
data.to_csv("./var_ds/syntax.csv")

{'word_ord': 35, 'simpl_synt': 21, 'verb_cats': 17, 'lexicon': 34, 'morphology': 8, 'phonology': 19, 'nom_synt': 5, 'comp_sent': 1}
